# Pre-processing of raster dataset for the uploaded to JEDI

The following notbooks set up to help raster-data-processing for the uploaded to JEDI
https://jedi.discomap.eea.europa.eu/Search

Requirements for raster upload to JEDI: 
- pixel sizes in (10m, 100, 1000m or 10 000m) 
- projection: LAEA -EPSG3035 
- datatype: INT8,UINT8,INT16 ant UINT16.
- pixel nbit proberty:  NO nbits--> remove NBITS 
- Compression:  -co COMPRESS=JPEG -co TILED=YES
- format: *.tif


Every raster in the zip will be added as an attribute to the dimension.
You can exclude values inside the JEDI website under the field options. Provide a value seperated list of all values you wish to exclude.
The exclude will only work for INT8,UINT8,INT16 ant UINT16.

So, there is a workaround in place:
1. Create a zip with the same tifs but these must be clipped to a small area.
This way the uploaded zip is small but enough to populate the schema in the db.
2. When the dim is created (but not completed) send us the path to the real zip file and we will take care of it



## (Tool 1) Layerstack (multiband raster) to single band raster / folder:

## (Tool 1 -A) small subset to prepare the jedi uploade *using a small clip


In [22]:
# tool1 layerstack to single band
import os, sys 
import arcpy
from arcpy.sa import *
from pathlib import Path
import os
import sys
from datetime import datetime
import multiprocessing
import glob
import logging
import zipfile
import shutil  ## for zipping


# set logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(message)s')
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)
logger.addHandler(ch)

# set data folder
sys.path.append(Path(os.getcwd()).parent.joinpath('src').as_posix())
sys.path.append(Path(os.getcwd()).joinpath('src').as_posix())
from constants import dir_signature

###################################################################################################################################### SET
base_path = os.path.join(dir_signature, 'SOC_JRC')
base_path_input = os.path.join(base_path, 'input') # , 'SOC_g_m2_90-21')   ###inputs
#outzip = OutRas+'zip'
zip_folder = r'L:\f02_data\SOC_JRC\output\jedi'
aoi_folder = r'L:\f02_data\SOC_JRC\input\aoi'
aoi_folder_out =r'L:\f02_data\SOC_JRC\output\jedi\aoi'


# To allow overwriting outputs change overwriteOutput option to True.
arcpy.env.overwriteOutput = True
first_year = 1990  # to put the year into the raster name 
###################################################################################################################################### SET
print ("START.................")

#InRaster = sys.argv[1] # like c:\\some\\path\\raster.ext
InRaster = os.path.join(dir_signature, 'SOC_JRC', 'input', 'SOC_g_m2_90_21_epsg3035.tif')
#InRaster = os.path.join(dir_signature, 'SOC_JRC', 'input', 'aoi','sog_jrc_inside_aoi.tif')  ##for testing

print(InRaster)



##check projection: 3035 and reprocect


##clip raster to small AOI 
#https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/clip.htm
#arcpy.management.Clip(in_raster, rectangle, out_raster, {in_template_dataset}, {nodata_value}, {clipping_geometry}, {maintain_clipping_extent})

#aoi (luxembourg)
extent = "3015500 2933800 4014700 4071300"
OutRaster= aoi_folder+"\\"+"SOC_g_m2_90_21_epsg3035.tif"
arcpy.Clip_management(InRaster, extent, OutRaster)
InRaster = OutRaster


# get a list of the bands that make up the raster
arcpy.env.workspace = InRaster
bRng = arcpy.ListRasters()
print (bRng)


year = first_year
for ThisBnd in bRng:
#     # loop through the bands and export each one with CopyRaster
    InBand  = '{}\\{}'.format(InRaster,ThisBnd)
    bndDesc = arcpy.Describe(InBand)
    #NoData  = bndDesc.noDataValue 
    
    InSplit = os.path.splitext(InRaster) # split the image name and extension
    # output file name is c:\\some\\path\\raster_Band_X.ext
    band_name= str(year)
    #OutRas  = '{}_{}{}'.format(InSplit[0],ThisBnd,InSplit[1])
    OutRas  = '{}_{}{}'.format(InSplit[0],band_name,InSplit[1])

    pixel_type_info ='16_BIT_SIGNED'
    #format_info = 'COG'
    NoData_info = -999
    #https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/copy-raster.htm
        ##Usage: CopyRaster_management(
    ##			in_raster, out_rasterdataset, {config_keyword}, {background_value}, 
    ##			{nodata_value}, {NONE | OneBitTo8Bit}, {NONE | ColormapToRGB}, 
    ##			{1_BIT | 2_BIT | 4_BIT | 8_BIT_UNSIGNED | 8_BIT_SIGNED | 16_BIT_UNSIGNED | 
    ##			16_BIT_SIGNED | 32_BIT_UNSIGNED | 32_BIT_SIGNED | 32_BIT_FLOAT | 64_BIT}, 
    ##			{NONE | ScalePixelValue}, {NONE | RGBToColormap}, {TIFF | IMAGINE Image | 
    ##			BMP | GIF | PNG | JPEG | JPEG2000 | Esri Grid | Esri BIL | Esri BSQ | 
    ##			Esri BIP | ENVI | CRF | MRF}, {NONE | Transform}, {CURRENT_SLICE | ALL_SLICES}, {NO_TRANSPOSE | TRANSPOSE})

    
    
    head, tail = os.path.split(OutRas)
    raster_name_with_extension = tail
    OutRas=aoi_folder_out+"\\"+raster_name_with_extension.rsplit('.', maxsplit=1)[0]+'clip.tif'


    #arcpy.management.CopyRaster(in_raster, out_rasterdataset, {config_keyword}, {background_value}, {nodata_value}, {onebit_to_eightbit}, {colormap_to_RGB}, {pixel_type}, {scale_pixel_value}, {RGB_to_Colormap}, {format}, {transform}, {process_as_multidimensional}, {build_multidimensional_transpose})
    arcpy.CopyRaster_management(InBand,OutRas,nodata_value = NoData_info,pixel_type=pixel_type_info)#,format=format_info)


    ## zip raster file:
    #head, tail = os.path.split(OutRas)
    #raster_name_with_extension = tail
    #outzip=raster_name_with_extension.rsplit('.', maxsplit=1)[0]+'.zip'
    outzip=aoi_folder_out+"\\"+raster_name_with_extension.rsplit('.', maxsplit=1)[0]+'.zip'


    with zipfile.ZipFile(outzip, 'w') as f:#
        f.write(OutRas, os.path.basename(OutRas))

    year = year+1

#### next: zip all single zips again:

 #reading all zip files in zip folder:
 #https://note.nkmk.me/en/python-zipfile/
#shutil.make_archive(zip_folder+'\\dim\\'+'DIM_SOC_JRC_2019_2021','zip',zip_folder)

print ("done")

START.................
\\cwsfileserver\projects\lulucf\f02_data\SOC_JRC\input\SOC_g_m2_90_21_epsg3035.tif
['Band_1', 'Band_2', 'Band_3', 'Band_4', 'Band_5', 'Band_6', 'Band_7', 'Band_8', 'Band_9', 'Band_10', 'Band_11', 'Band_12', 'Band_13', 'Band_14', 'Band_15', 'Band_16', 'Band_17', 'Band_18', 'Band_19', 'Band_20', 'Band_21', 'Band_22', 'Band_23', 'Band_24', 'Band_25', 'Band_26', 'Band_27', 'Band_28', 'Band_29', 'Band_30', 'Band_31', 'Band_32']
done


In [21]:
#print (InSplit[0])

head, tail = os.path.split(OutRas)
raster_name_with_extension = tail
out_raster_name=raster_name_with_extension.rsplit('.', maxsplit=1)[0]+'.tif'
print(out_raster_name)


SOC_g_m2_90_21_epsg3035_1990clip.tif


## (Tool 1 -B) FULL production of JEDI upload:

In [24]:
# tool1 layerstack to single band
import os, sys 
import arcpy
from arcpy.sa import *
from pathlib import Path
import os
import sys
from datetime import datetime
import multiprocessing
import glob
import logging
import zipfile
import shutil  ## for zipping


# set logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(message)s')
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)
logger.addHandler(ch)

# set data folder
sys.path.append(Path(os.getcwd()).parent.joinpath('src').as_posix())
sys.path.append(Path(os.getcwd()).joinpath('src').as_posix())
from constants import dir_signature

###################################################################################################################################### SET
base_path = os.path.join(dir_signature, 'SOC_JRC')
base_path_input = os.path.join(base_path, 'input') # , 'SOC_g_m2_90-21')   ###inputs
#outzip = OutRas+'zip'
zip_folder = r'L:\f02_data\SOC_JRC\output\jedi'

# To allow overwriting outputs change overwriteOutput option to True.
arcpy.env.overwriteOutput = True
first_year = 1990  # to put the year into the raster name 
###################################################################################################################################### SET
print ("START.................")

#InRaster = sys.argv[1] # like c:\\some\\path\\raster.ext
InRaster = os.path.join(dir_signature, 'SOC_JRC', 'input', 'SOC_g_m2_90_21_epsg3035.tif')
#InRaster = os.path.join(dir_signature, 'SOC_JRC', 'input', 'aoi','sog_jrc_inside_aoi.tif')  ##for testing

print(InRaster)


##check projection: 3035 --if not  re-projcect raster to 3035






# get a list of the bands that make up the raster
arcpy.env.workspace = InRaster
bRng = arcpy.ListRasters()
print (bRng)


year = first_year
for ThisBnd in bRng:
#     # loop through the bands and export each one with CopyRaster
    InBand  = '{}\\{}'.format(InRaster,ThisBnd)
    bndDesc = arcpy.Describe(InBand)
    #NoData  = bndDesc.noDataValue 
    
    InSplit = os.path.splitext(InRaster) # split the image name and extension
    # output file name is c:\\some\\path\\raster_Band_X.ext
    band_name= str(year)
    #OutRas  = '{}_{}{}'.format(InSplit[0],ThisBnd,InSplit[1])
    OutRas  = '{}_{}{}'.format(InSplit[0],band_name,InSplit[1])

    pixel_type_info ='16_BIT_SIGNED'
    #format_info = 'COG'
    NoData_info = -999
    #https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/copy-raster.htm
        ##Usage: CopyRaster_management(
    ##			in_raster, out_rasterdataset, {config_keyword}, {background_value}, 
    ##			{nodata_value}, {NONE | OneBitTo8Bit}, {NONE | ColormapToRGB}, 
    ##			{1_BIT | 2_BIT | 4_BIT | 8_BIT_UNSIGNED | 8_BIT_SIGNED | 16_BIT_UNSIGNED | 
    ##			16_BIT_SIGNED | 32_BIT_UNSIGNED | 32_BIT_SIGNED | 32_BIT_FLOAT | 64_BIT}, 
    ##			{NONE | ScalePixelValue}, {NONE | RGBToColormap}, {TIFF | IMAGINE Image | 
    ##			BMP | GIF | PNG | JPEG | JPEG2000 | Esri Grid | Esri BIL | Esri BSQ | 
    ##			Esri BIP | ENVI | CRF | MRF}, {NONE | Transform}, {CURRENT_SLICE | ALL_SLICES}, {NO_TRANSPOSE | TRANSPOSE})


    #arcpy.management.CopyRaster(in_raster, out_rasterdataset, {config_keyword}, {background_value}, {nodata_value}, {onebit_to_eightbit}, {colormap_to_RGB}, {pixel_type}, {scale_pixel_value}, {RGB_to_Colormap}, {format}, {transform}, {process_as_multidimensional}, {build_multidimensional_transpose})
    arcpy.CopyRaster_management(InBand,OutRas,nodata_value = NoData_info,pixel_type=pixel_type_info)#,format=format_info)


    ## zip raster file:
    head, tail = os.path.split(OutRas)
    raster_name_with_extension = tail


    outzip=zip_folder+"\\"+raster_name_with_extension.rsplit('.', maxsplit=1)[0]+'.zip'


    with zipfile.ZipFile(outzip, 'w') as f:#
        f.write(OutRas, os.path.basename(OutRas))

    

    year = year+1

#### next: zip all single zips again:

 #reading all zip files in zip folder:
 #https://note.nkmk.me/en/python-zipfile/
#shutil.make_archive(zip_folder+'\\dim\\'+'DIM_SOC_JRC_2019_2021','zip',zip_folder)

print ("done")

START.................
\\cwsfileserver\projects\lulucf\f02_data\SOC_JRC\input\SOC_g_m2_90_21_epsg3035.tif
['Band_1', 'Band_2', 'Band_3', 'Band_4', 'Band_5', 'Band_6', 'Band_7', 'Band_8', 'Band_9', 'Band_10', 'Band_11', 'Band_12', 'Band_13', 'Band_14', 'Band_15', 'Band_16', 'Band_17', 'Band_18', 'Band_19', 'Band_20', 'Band_21', 'Band_22', 'Band_23', 'Band_24', 'Band_25', 'Band_26', 'Band_27', 'Band_28', 'Band_29', 'Band_30', 'Band_31', 'Band_32']
done
